# Predict last emotion of a dialogue from MELD dataset

In [1]:
# Set Wandb
import wandb

entity = 'IFT6759-ADV_PROJECT'
project = 'cnn-emotion-prediction'
secret_api_key = 'e3d9da3d2519364013b968011af022ee0bda0091' # 'your_api_key_here'

wandb.login(key=secret_api_key)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\allar\_netrc
wandb: Currently logged in as: toma-allary (toma-allary-universit-de-montr-al) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

### If features were not already processed through WaveLM & RoBERTa

In [2]:
import numpy as np
import pandas as pd
import os
from src.conversation_processing import ConversationProcessor_CNN
import torch
import pickle

# Limit to 50% of the total available memory on the GPU
torch.cuda.set_per_process_memory_fraction(0.8, device=0)

project_dir                 = os.getcwd()
MELD_folder                 = "train"
audio_data_folder           = "train_splits"
text_data_csv_filename      = "train_sent_emo.csv"
combined_audio_data_folder  = "train_combined"
combined_csv_filename       = "MELD_combined_v2.csv"
embeddings_plk_filename     = "final_tensor_conversation_v2"

myProcessor = ConversationProcessor_CNN(
    project_dir=project_dir,
    MELD_folder=MELD_folder,
    audio_data_folder=audio_data_folder,
    text_data_csv_filename=text_data_csv_filename,
    combined_audio_data_folder=combined_audio_data_folder,
    combined_csv_filename=combined_csv_filename,
    embeddings_plk_filename=embeddings_plk_filename)

print(myProcessor.BASE_DIR)

# Check if merged audio exist
if not os.listdir(myProcessor.COMBINED_DIR):
    print("Combined audio is empty. Creating combined audio.")
    # myProcessor.CombineAudio()
else:
    print("Combined audio is already present.")

if not os.path.exists(myProcessor.DATA_CSV_FILE):
    print("Combined csv does not exist, creating it..")
    # combined_df = myProcessor.Create_combined_csv()
else:
    combined_df = pd.read_csv(myProcessor.DATA_CSV_FILE)
    print("Combined csv is already present.")

if not os.path.exists(myProcessor.CODE_FILENAME) or not os.path.exists(myProcessor.CODE_LABEL_FILENAME):
    print("Combined features does not exist. Creating features dataset..")
    # myProcessor.Create_features_dataset(combined_df)
else:
    print("Combined features dataset is already present.")

with open(myProcessor.CODE_FILENAME, 'rb') as f:
    train_dataset_tensor = pickle.load(f)

train_dataset_targets = pd.read_csv(myProcessor.CODE_LABEL_FILENAME)

# Optionally confirm it's a tensor
print("dataset Tensor and CSV loaded.")

C:\Users\allar\miniconda3\envs\IFT6759\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ConversationProcessor initialized using device: cuda
C:\Users\allar\Documents\UdeM\IFT6759 AdvProject\Projet\IFT-6759-Advanced-ML-Project
Combined audio is already present.
Combined csv is already present.
Combined features dataset is already present.
dataset Tensor and CSV loaded.


In [3]:
print(f"Dataset shape: {train_dataset_tensor.shape}")
train_dataset_targets

Dataset shape: torch.Size([734, 128, 1536])


,Dialogue_ID,Last_Emotion
0,2,joy
1,3,neutral
2,4,neutral
3,5,surprise
4,6,anger
...,...,...
729,995,neutral
730,996,sadness
731,997,anger
732,998,surprise


## Split train set to have validation

In [4]:
import torch
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset

# Ratio for training
ratio_train = 0.8
batch_size = 32
max_batch_size = 4

# Convert string target into encoded classes
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(train_dataset_targets['Last_Emotion'].values)
# Convert labels into tensor
labels = torch.tensor(labels_encoded, dtype=torch.long)

# Merge features & target in a TensorDataset
dataset = TensorDataset(train_dataset_tensor, labels)

# Split the dataset into train and validation
train_size = int(ratio_train * len(dataset))
val_size = len(dataset) - train_size
train_data, val_data = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create DataLoader
train_loader = DataLoader(train_data, batch_size=max_batch_size, num_workers=1, pin_memory=False, shuffle=True)
val_loader = DataLoader(val_data, batch_size=max_batch_size, num_workers=1, pin_memory=False)


print(f"Train dataset shape: {len(train_loader.dataset)}")
print(f"Val dataset shape: {len(val_loader.dataset)}")

Train dataset shape: 587
Val dataset shape: 147


## Train ResNet architecture

In [5]:
import optuna

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
use_amp = torch.cuda.is_available()



class Args:
    lr: float = 0.0001
    epochs: int = 100
    optimizer = "adam"
    weight_decay = 0.0001


def TrainModel(model: torch.nn.Module, args: Args, optuna_trial: optuna.Trial = None):

    optimizer_delay = -(-batch_size // max_batch_size) # round up
    # after optimizer_delay steps, do backward. This emulate bigger batch_size

    criterion = torch.nn.CrossEntropyLoss()
    if use_amp:
        scaler = torch.amp.GradScaler('cuda')

    if args.optimizer == "sgd":
        optimizer = torch.optim.SGD(model.parameters(), lr=args.lr, weight_decay=args.weight_decay, momentum=0.9)
    elif args.optimizer == "adam":
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    elif args.optimizer == "adamw":
        optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    else:
        raise ValueError(f"Unknown optimizer {args.optimizer}")

    # Training and validation loop
    best_val_acc = 0
    pruned = False

    last_train_loss = 0
    last_val_acc = 0
    last_train_acc = 0
    for epoch in range(args.epochs):
        # --- Training ---
        model.train()
        total_loss = 0

        batch_it = 0
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            if use_amp:
                with torch.amp.autocast('cuda'):
                    outputs = model(batch_x)
                    loss = criterion(outputs, batch_y)

                scaler.scale(loss).backward()

                if batch_it % optimizer_delay == 0 or batch_it == len(train_loader) - 1:
                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad()
            else:
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                if batch_it % optimizer_delay == 0 or batch_it == len(train_loader) - 1:
                    optimizer.step()
                    optimizer.zero_grad()

            total_loss += loss.item()
            batch_it += 1

        train_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}")

        # --- Validation ---
        model.eval()
        val_correct = 0
        val_total = 0
        train_correct = 0
        train_total = 0
        with torch.no_grad():
            for batch_x, batch_y in train_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)

                outputs = model(batch_x)
                _, predicted = torch.max(outputs, 1)
                train_total += batch_y.size(0)
                train_correct += (predicted == batch_y).sum().item()

            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)

                outputs = model(batch_x)
                _, predicted = torch.max(outputs, 1)
                val_total += batch_y.size(0)
                val_correct += (predicted == batch_y).sum().item()

        val_accuracy = val_correct / val_total * 100
        train_accuracy = train_correct / train_total * 100
        print(f"Validation Accuracy: {val_accuracy:.2f}%")
        print(f"Train Accuracy: {train_accuracy:.2f}%")


        if optuna_trial is not None:
            # Optuna pruning
            optuna_trial.report(val_accuracy, step=epoch)
            # Prune if trial is not promising
            if optuna_trial.should_prune():
                pruned = True
                return train_loss, train_accuracy, val_accuracy, best_val_acc, pruned

        best_val_acc = max(best_val_acc, val_accuracy)
        last_train_loss = train_loss
        last_val_acc = val_accuracy
        last_train_acc = train_accuracy

        # Log metrics using the global context
        wandb.log({"train_acc": train_accuracy, "train_loss": train_loss, "val_acc": val_accuracy})

        # Clear cache after each epoch
        torch.cuda.empty_cache()

    return last_train_loss, last_train_acc, last_val_acc, best_val_acc, pruned

Using device: cuda


In [6]:
from datetime import datetime
from src.resnet_cnn import ResNetCNN
from src.cnn_classifier import CNNClassifier

import os
# This makes the GPU operations synchronous
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# torch.cuda.empty_cache()

model_type = 'resnet'
if model_type == 'resnet':
    model = ResNetCNN()
else:
    model = CNNClassifier()
model = model.to(device)

someArgs = Args()
run = wandb.init(
    entity=entity,
    project=project,
    name=f"{model_type}-emotion-prediction-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    config={
        "learning_rate": someArgs.lr,
        "architecture": model_type,
        "dataset": "MELD",
        "epochs": someArgs.epochs,
        "batch_size": batch_size,
    },
)

TrainModel(model, someArgs)


Epoch 1, Train Loss: 1.7324
Validation Accuracy: 46.94%
Train Accuracy: 44.63%
Epoch 2, Train Loss: 1.6051
Validation Accuracy: 46.94%
Train Accuracy: 44.63%
Epoch 3, Train Loss: 1.5910
Validation Accuracy: 46.94%
Train Accuracy: 44.63%
Epoch 4, Train Loss: 1.6074
Validation Accuracy: 46.94%
Train Accuracy: 44.63%
Epoch 5, Train Loss: 1.6000
Validation Accuracy: 46.94%
Train Accuracy: 44.63%
Epoch 6, Train Loss: 1.6020
Validation Accuracy: 46.94%
Train Accuracy: 44.63%
Epoch 7, Train Loss: 1.6006
Validation Accuracy: 46.94%
Train Accuracy: 44.63%
Epoch 8, Train Loss: 1.5920
Validation Accuracy: 46.94%
Train Accuracy: 44.63%
Epoch 9, Train Loss: 1.5901
Validation Accuracy: 46.94%
Train Accuracy: 44.63%
Epoch 10, Train Loss: 1.5970
Validation Accuracy: 46.94%
Train Accuracy: 44.63%
Epoch 11, Train Loss: 1.5794
Validation Accuracy: 46.94%
Train Accuracy: 44.63%
Epoch 12, Train Loss: 1.5796
Validation Accuracy: 46.94%
Train Accuracy: 44.63%
Epoch 13, Train Loss: 1.5703
Validation Accuracy:

(0.04021173593949298,
 99.31856899488926,
 20.408163265306122,
 46.93877551020408,
 False)

# Hyperparameter search

In [7]:
import optuna
from optuna.pruners import MedianPruner
from optuna.integration.wandb import WeightsAndBiasesCallback
from src.cnn_classifier import CNNClassifier
from src.resnet_cnn import ResNetCNN
from datetime import datetime
import time

time_elapsed_trial = []

# 1. Define an objective function to be maximized.
def objective(trial):
    someArgs = Args()

    # Track time
    print(f"Starting trial {len(time_elapsed_trial)}")
    if len(time_elapsed_trial) > 0:
        mean_time = 0
        for i in time_elapsed_trial:
            mean_time += i
        mean_time /= len(time_elapsed_trial)
        print(f"Mean time per trial: {mean_time}")
    start_time = time.time()

    # === 1. Suggest Hyperparameters ===
    someArgs.lr = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    someArgs.epochs = 150
    model_type = 'cnn' # trial.suggest_categorical("architecture", ["resnet", "cnn"])
    activation_fn = trial.suggest_categorical("activation_fn", ["relu", "leaky_relu", "gelu"])
    someArgs.optimizer = trial.suggest_categorical("optimizer", ["adam", "sgd", "adamw"])
    someArgs.weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)

    # Train
    if model_type == 'resnet':
        model = ResNetCNN(activation_function=activation_fn)
    else:
        model = CNNClassifier(activation_function=activation_fn)
    model = model.to(device)

    run = wandb.init(
        entity=entity,
        project=project,
        name=f"{model_type}-emotion-prediction-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
        config={
            "learning_rate": someArgs.lr,
            "architecture": model_type,
            "dataset": "MELD",
            "epochs": someArgs.epochs,
            "batch_size": batch_size,
        },
        reinit=True,
    )

    train_loss, train_acc, val_acc, best_val_acc, pruned = TrainModel(model, someArgs, optuna_trial=trial)

    run.finish()
    if pruned:
        end = time.time()
        length = end - start_time
        time_elapsed_trial.append(length)
        raise optuna.exceptions.TrialPruned()


    end = time.time()
    length = end - start_time
    time_elapsed_trial.append(length)
    return best_val_acc

# Log optuna result in wandb
wandb_callback = WeightsAndBiasesCallback(
    metric_name="val_acc",
    wandb_kwargs={"entity": entity, "project": project},
)
# Optuna tuning and pruning
pruner = MedianPruner(n_warmup_steps=5)
study = optuna.create_study(
    direction='maximize',
    pruner=pruner
)
time_to_run = 1 # this is hours
time_to_run = int(time_to_run * 3600) #convert in seconds
study.optimize(objective, timeout=time_to_run, callbacks=[wandb_callback], n_jobs=1)

C:\Users\allar\AppData\Local\Temp\ipykernel_15324\3852091136.py:70: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  wandb_callback = WeightsAndBiasesCallback(


[I 2025-04-07 21:00:51,685] A new study created in memory with name: no-name-fa0dde68-7f8c-408d-b4d1-b4f1b46877d3


Starting trial 0


Epoch 1, Train Loss: 1.7462
Validation Accuracy: 42.86%
Train Accuracy: 46.00%
Epoch 2, Train Loss: 1.6153
Validation Accuracy: 44.90%
Train Accuracy: 45.14%
Epoch 3, Train Loss: 1.5630
Validation Accuracy: 44.90%
Train Accuracy: 45.14%
Epoch 4, Train Loss: 1.5373
Validation Accuracy: 44.90%
Train Accuracy: 45.14%
Epoch 5, Train Loss: 1.5032
Validation Accuracy: 44.90%
Train Accuracy: 45.14%
Epoch 6, Train Loss: 1.5161
Validation Accuracy: 44.22%
Train Accuracy: 54.00%
Epoch 7, Train Loss: 1.4135
Validation Accuracy: 44.90%
Train Accuracy: 45.14%
Epoch 8, Train Loss: 1.3152
Validation Accuracy: 44.90%
Train Accuracy: 45.49%
Epoch 9, Train Loss: 1.0893
Validation Accuracy: 45.58%
Train Accuracy: 66.27%
Epoch 10, Train Loss: 0.9221
Validation Accuracy: 34.01%
Train Accuracy: 74.62%
Epoch 11, Train Loss: 0.8691
Validation Accuracy: 25.17%
Train Accuracy: 93.02%
Epoch 12, Train Loss: 0.7405
Validation Accuracy: 38.10%
Train Accuracy: 89.44%
Epoch 13, Train Loss: 0.5728
Validation Accuracy:

train_acc,▁▁▂▁▇███████████████████████████████████
train_loss,███▇▆▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▇██▃▅█▄█▇▆▆▆▅▆▆▃▆▆▇▁▄▆▅█▆▆▇▄▅▅▇▆▇▇▆▃▄█▆█
train_acc,100
train_loss,0.01645
val_acc,43.53741


...

Starting trial 3
Mean time per trial: 907.7556254863739


learning_rate,▁
val_acc,▁
weight_decay,▁
activation_fn,relu
learning_rate,3e-05
optimizer,adamw
val_acc,46.93878
weight_decay,2e-05


Epoch 1, Train Loss: 1.8382
Validation Accuracy: 44.90%
Train Accuracy: 45.14%
Epoch 2, Train Loss: 1.6396
Validation Accuracy: 44.90%
Train Accuracy: 45.14%
Epoch 3, Train Loss: 1.5824
Validation Accuracy: 44.90%
Train Accuracy: 45.14%
Epoch 4, Train Loss: 1.5598
Validation Accuracy: 44.90%
Train Accuracy: 45.14%
Epoch 5, Train Loss: 1.5282
Validation Accuracy: 44.90%
Train Accuracy: 45.14%
Epoch 6, Train Loss: 1.4739
Validation Accuracy: 44.22%
Train Accuracy: 48.89%
Epoch 7, Train Loss: 1.4353
Validation Accuracy: 44.90%
Train Accuracy: 45.14%
Epoch 8, Train Loss: 1.3738
Validation Accuracy: 44.90%
Train Accuracy: 46.17%
Epoch 9, Train Loss: 1.2339
Validation Accuracy: 44.90%
Train Accuracy: 48.89%
Epoch 10, Train Loss: 1.1661
Validation Accuracy: 43.54%
Train Accuracy: 63.54%
Epoch 11, Train Loss: 1.0169
Validation Accuracy: 44.90%
Train Accuracy: 81.26%
Epoch 12, Train Loss: 0.9778
Validation Accuracy: 44.22%
Train Accuracy: 69.17%
Epoch 13, Train Loss: 0.8543
Validation Accuracy:

train_acc,▁▁▁▁████████████████████████████████████
train_loss,█▇▆▅▄▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,██▇▇▇█▅▆▅▆▆▆▆▆▄▆▅▆▇▂▅▆▄▅▂▃▅▃▆▃▁▄▅▂▃▇▁▄▃▃
train_acc,100
train_loss,0.01516
val_acc,41.4966


[I 2025-04-07 22:01:30,107] Trial 3 finished with value: 44.89795918367347 and parameters: {'learning_rate': 0.0004056095056921135, 'activation_fn': 'leaky_relu', 'optimizer': 'adamw', 'weight_decay': 8.564543990221551e-06}. Best is trial 2 with value: 46.93877551020408.


In [8]:
model.eval()

val_total = 0
val_correct = 0
for batch_x, batch_y in val_loader:
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    outputs = model(batch_x)
    _, predicted = torch.max(outputs, 1)
    val_total += batch_y.size(0)
    val_correct += (predicted == batch_y).sum().item()

    print("#############################################################")
    print("PREDICTION")
    print(predicted)
    print("TARGET")
    print(batch_y)

print(f"accuracy: {val_correct / val_total * 100:.2f}%")

NameError: name 'model' is not defined

## CNN nest model so far
Trial 3 finished with value: 44.89795918367347 and parameters: {'learning_rate': 0.0004056095056921135, 'activation_fn': 'leaky_relu', 'optimizer': 'adamw', 'weight_decay': 8.564543990221551e-06}. Best is trial 2 with value: 46.93877551020408.